In [1]:
import pandas as pd
import numpy as np

from scipy.integrate import quad
from scipy.optimize import minimize


from multiprocessing import Pool
from functools import partial
import time
from datetime import datetime

from typing import Dict

import json
import logging
import sys

In [2]:
sample = json.load(open('data/snls_snia.json'))

df = pd.DataFrame(sample)

In [3]:
def Dc(z, Omega_l, Omega_m, Omega_k, w):
    def f(z, Omega_l, Omega_m, Omega_k, w):
        return (np.sqrt(Omega_l*(1 + z)**(-3 - 3*w) + Omega_m*(1 + z)**3 + Omega_k*(1 + z)**2 + 9.2e-5*(1 + z)**4))**(-1)
    return quad(f, 0, z, args = (Omega_l, Omega_m, Omega_k, w))[0]

def Dt(z, Omega_l, Omega_m, Omega_k, w):
    if Omega_k > 0:
        return np.sin(np.sqrt(Omega_k)*Dc(z, Omega_l, Omega_m, Omega_k, w))/np.sqrt(Omega_k)
    elif Omega_k == 0:
        return Dc(z, Omega_l, Omega_m, Omega_k, w)
    elif Omega_k:
        return np.sinh(np.sqrt(abs(Omega_k))*Dc(z, Omega_l, Omega_m, Omega_k, w))/np.sqrt(abs(Omega_k))

def Dl(z, Omega_l, Omega_m, Omega_k, w):
    return (1 + z) * Dt(z, Omega_l, Omega_m, Omega_k, w)
               
def mu(z, Omega_l, Omega_m, Omega_k, w, Hubble):
    c = 3e5 #(km/s)
    return 5*np.log10(Dl(z, Omega_l, Omega_m, Omega_k, w)) + 25 + 5*np.log10(c/ Hubble)

def chi2_i(mu_o, sigma_o, z_obs, Omega_l, Omega_m, Omega_k, w, Hubble):
    return ((mu(z_obs, Omega_l, Omega_m, Omega_k, w, Hubble) - mu_o)**2)/sigma_o**2

def chi2(Omega_l, Omega_m, Omega_k, w, Hubble, df): 
    values = []
    for idx in df.index:
        values.append(chi2_i(df.mu[idx], df.sigma[idx], df.z[idx], Omega_l, Omega_m, Omega_k, w, Hubble))
    return sum(values)

def ratio(Omega_l, Omega_m, Omega_k, w, Hubble, df, chi2null):
    return chi2(Omega_l, Omega_m, Omega_k, w, Hubble, df) - chi2null

In [4]:
def _zip_vars(fixed: dict, x) -> dict:
    full_parameters = ['Omega_l', 'Omega_m', 'Omega_k', 'w',  'Hubble']
    variable_parameters = []
    for param in full_parameters:
        if param not in list(fixed.keys()):
            variable_parameters.append(param)
            
    _vars = dict(zip(variable_parameters, x))
    return _vars

In [19]:
def h(x, fixed, df):
    """
    This is a wrapper for the chi2 function. 
    A workaround to the fact that scipy.optimize.minimize uses positional arguments
    
        Given a dict of fixed variables, this fuction enables minimize to pass an array of arguments
    """
    
     # Creates dictionary of variables to be optimized
    _vars = _zip_vars(fixed, x)

    # fixed['df'] = df
    g = partial(chi2, df = df, **fixed) # Creates partial function fixing variables set by `fixed`
        
    # Unzips `_vars` as named args to `g`
    return g(**_vars) 

In [27]:
config = json.load(open('config/likelihood_ratio_config.json'))

config['df'] = df

grid_size = config['grid_sizes'][0]

def generate_grid(config, grid_size):
    fixed = {}
    x0 = config['minimization_seed']
    for i in np.linspace(config['grid'][config['free'][0]][0], config['grid'][config['free'][0]][1], grid_size):
        fixed[config['free'][0]] = i
        for j in np.linspace(config['grid'][config['free'][1]][0], config['grid'][config['free'][1]][1], grid_size):
            fixed[config['free'][1]] = j
            
            minimum = minimize(h, x0 = x0, args = (fixed, config['df']), method = 'Nelder-Mead', tol = 1e-6)
            x0 = minimum.x
            
            _vars = _zip_vars(fixed, minimum.x)
            
            keys = ['Omega_l', 'Omega_m', 'Omega_k', 'w', 'Hubble']
            values = [fixed.get(key,  _vars.get(key)) for key in keys]
            
            yield tuple(values)
            
# for i,j,k,l,m in generate_grid(config, grid_size):
#     print(i,j,k,l,m)

In [31]:
# Optimizing every variable to find null hypothesis

fixed = {'w':-1}

x0 = [0.76, 0.24, 0.02, 71] #['Omega_l', 'Omega_m', 'Omega_k', 'w',  'Hubble']

minimum = minimize(h, x0 = x0, args = (fixed, df), method = 'Nelder-Mead', tol = 1e-6, options = {'maxiter': 10000})

In [32]:
minimum.x

array([ 5.42698378e-02, -2.43879477e-01,  4.82269089e-01,  1.29495160e+02])

In [33]:
minimum

 final_simplex: (array([[ 5.42698378e-02, -2.43879477e-01,  4.82269089e-01,
         1.29495160e+02],
       [ 5.42698343e-02, -2.43879480e-01,  4.82269092e-01,
         1.29495160e+02],
       [ 5.42698338e-02, -2.43879480e-01,  4.82269093e-01,
         1.29495160e+02],
       [ 5.42698340e-02, -2.43879480e-01,  4.82269093e-01,
         1.29495160e+02],
       [ 5.42698320e-02, -2.43879481e-01,  4.82269095e-01,
         1.29495161e+02]]), array([110.79524005, 110.79524005, 110.79524005, 110.79524005,
       110.79524005]))
           fun: 110.79524004671357
       message: 'Optimization terminated successfully.'
          nfev: 1503
           nit: 902
        status: 0
       success: True
             x: array([ 5.42698378e-02, -2.43879477e-01,  4.82269089e-01,  1.29495160e+02])